In [ ]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

In [2]:
pip install torch transformers peft datasets

  Using cached attrs-25.3.0-py3-none-any.whl.metadata (10 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 MB 4.8 MB/s  0:00:15m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 5.5 MB/s  0:00:02 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 5.5 MB/s  0:00:00 eta 0:00:01
Using cached attrs-25.3.0-py3-none-any.whl (63 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.2/31.2 MB 5.0 MB/s  0:00:06m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 5.3 MB/s  0:00:01 eta 0:00:01
Using cached mpmath-1.3.0-py3-none-any.whl (536 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 5.9 MB/s  0:00:00 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 4.9 MB/s  0:00:02 eta 0:00:01
Using cached pytz-2025.2-py2.py3-n

In [1]:
from datasets import load_dataset
dataset = load_dataset("SetFit/emotion")

Repo card metadata block was not found. Setting CardData to empty.


In [2]:
train_data = dataset["train"]
test_data = dataset["test"]

In [3]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

In [4]:
# Tokenization Function
def tokenize_function(batch):
    return tokenizer(batch["text"], padding="max_length", truncation=True)

# Apply tokenization
train_dataset = train_data.map(tokenize_function, batched=True)
test_dataset = test_data.map(tokenize_function, batched=True)

# Convert to PyTorch format
train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])
train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])



Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [ ]:
from transformers import AutoModelForSequenceClassification

# load BERT model (6 output classes)
base_model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=6)

In [ ]:
print(base_model)

In [ ]:
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, TaskType

# Define LoRA Configuration
lora_config = LoraConfig(
    r=8, # Low-rank adaptation dimension
    lora_alpha=32, # Scaling Factor
    lora_dropout=0.05, # Dropout rate
    target_modules=["query", "value"] # Apply LoRA to self-attention layers only
)

# Prepare model for LoRA
base_model = prepare_model_for_kbit_training(base_model)

# Convert model into LoRA-enabled model
peft_model = get_peft_model(base_model, lora_config)

# Print trainable parameters
peft_model.print_trainable_parameters()

In [ ]:
from transformers import TrainingArguments, Trainer
training_args = TrainingArguments(# configuration class that defines how training should happen
    output_dir= "./model_checkpoints", # Where to save model
    num_train_epochs=1, # Train for 3 epochs
    per_device_train_batch_size=16, # 16 samples per GPU/CPU
    evaluation_strategy="epoch", # Evaluate after every epoch
    save_strategy="epoch",   # Save model after each epoch
    logging_steps=10,   # Log training metrics every 10 steps
    load_best_model_at_end=True, # Automatically load best checkpoint
    fp16=True   # Use mixed precision for faster training (if GPU supports it)
)

# A high-level class that automates training, evaluation, and saving models.
# It wraps around your model and dataset, handling:

# Training loops.
# Evaluation during training.
# Model saving & checkpointing.

trainer = Trainer(
    model=peft_model,  # LoRA fine-tuned model
    args=training_args,  # Training settings
    train_dataset=train_dataset,  # Training data
    eval_dataset=test_dataset,  # Test data
    tokenizer=tokenizer  # Tokenizer for processing text,
)

trainer.train()

In [ ]:
from transformers import BitsAndBytesConfig

# Apply 4-bit quantization to reduce memory
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,    # Enable 4-bit quantization
    bnb_4bit_compute_dtype=torch.float16, # Compute in float16
    bnb_4bit_use_double_quant=True,   # Double quantization for efficiency
)

# Load quantized BERT model
base_model = AutoModelForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels=6,
    quantization_config=bnb_config
)

# Apply LoRA on top of QLoRA
peft_model = get_peft_model(base_model, lora_config)

In [ ]:
from transformers import pipeline

# Load fine-tuned model from HF Hub
emotion_classifier = pipeline("text-classification", model="your-hf-username/bert-emotion-lora")

# Test on new sentences
print(emotion_classifier("I am so happy today!"))
print(emotion_classifier("I feel terrible and sad."))